<h1><center><b><font color='dark blue'>Passport MRZ Scanner   </font></b></center> </h1>

In [1]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25891 sha256=010e48568d55c37aaacd7b53d0358d779e09a6f7f172eeb6bdc2c4d21e647f5b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\85\cf\3a\e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


You should consider upgrading via the 'F:\smart-id-extraction\smartid-env\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install readmrz

<h3><b><font color='darkblue'>Imports</font></b></h3>

In [2]:
import cv2
import os
import numpy as np
import sys
import imutils
from imutils.contours import sort_contours
from PIL import Image
import pytesseract as tess
import matplotlib.pyplot as plt
from PIL import Image
from readmrz import MrzDetector, MrzReader

Iterate over all images and auto rotate them to normal position and save them at another directory

In [ ]:
for i in os.listdir("/kaggle/input/passport-images/images"):
    print(i)
    img = Image.open("/kaggle/input/passport-images/images/"+i)
    #img.save("C:\\Users\\Abduallah Hussien\\Desktop\\Cyshield Tasks\\Assignment\\Computer vision\\Task1\\images at normal position\\{}.png".format(i.split('.')[0]))

Take one image as a sample to get the output image after each preprocessing step

In [ ]:
detector = MrzDetector()
reader = MrzReader()
# 🔹 1. Replace with your local image path
image = r"C:\Users\YourUsername\Desktop\passport_sample.jpg"  # ← Change this
#plt.title('Original Image')
plt.imshow(image)

Image smoothing:
It is useful for removing noise. It actually removes high frequency content (eg: noise, edges) from the image.

In [ ]:
resized = detector.resize(image)
smoothed = detector.smooth(resized)

plt.title('Smoothed Image')
plt.imshow(smoothed)
plt.axis('off');

Black hat:
Operation is used to enhance dark objects of interest in a bright background.

In [ ]:
dark = detector.find_dark_regions(smoothed)
plt.title('Black-hat image')
plt.imshow(dark)
plt.axis('off');

I used two methods of thresholding
- thresholding using MrzDetector object
- thresholding using OpenCV

In [ ]:
def threshold1(dark):
    thresh = detector.apply_threshold(dark)

    #plt.title('Otsu\'s thresholding method')
    #plt.imshow(thresh)
    #plt.axis('off');
    return thresh

In [ ]:
def threshold2(dark):
    # compute the Scharr gradient of the blackhat image and scale the
    # result into the range [0, 255]
    grad = cv2.Sobel(dark, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    grad = np.absolute(grad)
    (minVal, maxVal) = (np.min(grad), np.max(grad))
    grad = (grad - minVal) / (maxVal - minVal)
    grad = (grad * 255).astype("uint8")
    # initialize a rectangular and square structuring kernel
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(25, 7))
            # apply a closing operation using the rectangular kernel to close
            # gaps in between letters -- then apply Otsu's thresholding method
    sqKernel = cv2.getStructuringElement(
                cv2.MORPH_RECT,
                (21, 21)
            )
    grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, rectKernel)
    thresh = cv2.threshold(grad, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            # cv2.imshow("Rect Close", thresh)

            # perform another closing operation, this time using the square
            # kernel to close gaps between lines of the MRZ, then perform a
            # series of erosions to break apart connected components
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
    thresh = cv2.erode(thresh, None, iterations=2)
    #plt.imshow(thresh)
    return thresh

In [ ]:
thresh = threshold2(dark)

Get the coordinates of expected Mrz

In [ ]:
y, y1, x, x1 = detector.find_coordinates(thresh, smoothed)
print(y, y1, x, x1)

Calculate width and height to draw our boundry box

In [ ]:
w = x1 - x
h = y1 - y

In [ ]:
fig, ax = plt.subplots()
ax.imshow(resized)

rect = plt.Rectangle((x, y), w, h, linewidth=1, edgecolor='g', facecolor='none')
ax.add_patch(rect)

plt.title('Machine readable zone')
plt.axis('off')
plt.show();

In [ ]:
plt.imshow(resized[y:y1, x:x1])
plt.axis('off');

Get the mrz code

In [ ]:
code = reader.read_mrz(resized[y:y1, x:x1])
print(code)

Get all traveler information from the mrz code

In [ ]:
reader.get_fields(code)

In [ ]:


# 📚 Imports
import cv2
import os
import numpy as np
import imutils
from imutils.contours import sort_contours
from PIL import Image
import matplotlib.pyplot as plt
from readmrz import MrzDetector, MrzReader
import json
from pathlib import Path

# 📁 Setup
image_path = "tests.jpeg"  # <-- Your local image (must be in the same folder)
output_folder = Path("ayuish")
output_folder.mkdir(parents=True, exist_ok=True)

# 🧠 Initialize detector and reader
detector = MrzDetector()
reader = MrzReader()

# 🔍 Load and process the image
print(f"Processing: {image_path}")
image = detector.read(image_path)
resized = detector.resize(image)
smoothed = detector.smooth(resized)
dark = detector.find_dark_regions(smoothed)

# 🔎 Threshold using method 2 (Scharr + Morphology)
grad = cv2.Sobel(dark, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
grad = np.absolute(grad)
grad = (grad - np.min(grad)) / (np.max(grad) - np.min(grad))
grad = (grad * 255).astype("uint8")

rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 7))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 21))

grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, rectKernel)
thresh = cv2.threshold(grad, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
thresh = cv2.erode(thresh, None, iterations=2)

# 🧭 Find MRZ coordinates
try:
    y, y1, x, x1 = detector.find_coordinates(thresh, smoothed)
    w = x1 - x
    h = y1 - y
except Exception as e:
    print(f"❌ Could not find MRZ in image: {e}")
    raise

# 📦 Crop MRZ
mrz_crop = resized[y:y1, x:x1]

# 🧾 Extract MRZ Code and Fields
code = reader.read_mrz(mrz_crop)
fields = reader.get_fields(code)

# 🖼️ Save MRZ crop
Image.fromarray(mrz_crop).save(output_folder / "mrz_crop.jpg")

# 📝 Save results to JSON
result = {
    "image_name": "test.jpg",
    "mrz_code": code,
    "coordinates": {
        "x": int(x), "x1": int(x1), "y": int(y), "y1": int(y1),
        "width": int(w), "height": int(h)
    },
    "fields": fields
}

with open(output_folder / "result.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# ✅ Show result
plt.imshow(mrz_crop)
plt.title("MRZ Crop")
plt.axis('off')
plt.show()

print("✅ Extraction complete! JSON and MRZ crop saved in 'ayuish' folder.")
